**PreProcessing**

**1. Resize**

In [5]:
from PIL import Image
import os

# Path folder yang berisi 5 kelas
parent_folder_path = "./Dataset/for CNN/train"

# Path folder untuk menyimpan gambar yang sudah diresize
output_folder = "./Dataset/for CNN/train 80"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop melalui setiap subfolder (kelas) dalam parent folder
for class_name in os.listdir(parent_folder_path):
    class_folder_path = os.path.join(parent_folder_path, class_name)
    if os.path.isdir(class_folder_path):
        # Membuat folder untuk kelas di dalam folder Resize
        class_output_folder = os.path.join(output_folder, class_name)
        if not os.path.exists(class_output_folder):
            os.makedirs(class_output_folder)

        # Memeriksa jumlah gambar sebelum resizing
        num_images_before = len(os.listdir(class_folder_path))
        print(f"Jumlah gambar dalam folder {class_folder_path}: {num_images_before}")

        # Loop melalui semua file dalam folder kelas
        for file_name in os.listdir(class_folder_path):
            if file_name.endswith((".jpg", ".jpeg", ".png", ".JPG")):
                image_path = os.path.join(class_folder_path, file_name)

                # Load gambar
                original_image = Image.open(image_path)

                # Resize gambar
                resized_image = original_image.resize((80, 80))

                # Simpan gambar yang sudah diresize
                output_path = os.path.join(class_output_folder, file_name)
                resized_image.save(output_path)

        # Memeriksa jumlah gambar setelah resizing
        num_images_after = len(os.listdir(class_output_folder))
        print(f"Jumlah gambar yang diresize untuk kelas {class_name}: {num_images_after} dari {num_images_before}")


Jumlah gambar dalam folder ./Dataset/for CNN/train\Busuk: 111
Jumlah gambar yang diresize untuk kelas Busuk: 111 dari 111
Jumlah gambar dalam folder ./Dataset/for CNN/train\Matang: 112
Jumlah gambar yang diresize untuk kelas Matang: 112 dari 112
Jumlah gambar dalam folder ./Dataset/for CNN/train\Mentah: 112
Jumlah gambar yang diresize untuk kelas Mentah: 112 dari 112
Jumlah gambar dalam folder ./Dataset/for CNN/train\Menuju Matang: 112
Jumlah gambar yang diresize untuk kelas Menuju Matang: 112 dari 112
Jumlah gambar dalam folder ./Dataset/for CNN/train\Terlalu Matang: 112
Jumlah gambar yang diresize untuk kelas Terlalu Matang: 112 dari 112


**2. Rotation**

In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

# Fungsi untuk melakukan rotasi pada gambar
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, M, (w, h))
    return rotated_image

# Fungsi untuk melakukan augmentasi rotasi pada dataset
def augment_dataset_with_rotation(input_folder, output_folder, angles=[30, 60, 90, 120, 150]):
    # Pastikan folder output ada atau buat jika belum ada
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Daftar nama kelas
    classes = os.listdir(input_folder)
    
    # Loop untuk setiap kelas
    for class_name in classes:
        print(f"Augmentasi kelas: {class_name}")
        class_folder = os.path.join(input_folder, class_name)
        output_class_folder = os.path.join(output_folder, class_name)
        
        # Buat folder kelas di folder output jika belum ada
        if not os.path.exists(output_class_folder):
            os.makedirs(output_class_folder)
        
        # Daftar semua gambar di folder kelas
        images = os.listdir(class_folder)
        
        # Loop untuk setiap gambar
        for img_name in tqdm(images, desc=f"Kelas: {class_name}", position=0, leave=True):
            # Baca gambar
            img_path = os.path.join(class_folder, img_name)
            img = cv2.imread(img_path)
            
            # Simpan gambar asli
            output_img_path = os.path.join(output_class_folder, img_name)
            cv2.imwrite(output_img_path, img)
            
            # Augmentasi dengan rotasi
            for angle in angles:
                rotated_img = rotate_image(img, angle)
                output_img_name = os.path.splitext(img_name)[0] + f"_rotated_{angle}.jpg"
                output_img_path = os.path.join(output_class_folder, output_img_name)
                
                # Simpan gambar yang dirotasi
                cv2.imwrite(output_img_path, rotated_img)

# Path ke folder dataset asli dan folder output
input_folder = './Dataset/1. Resize'
output_folder = './Dataset/3. Rotation'

# Panggil fungsi untuk melakukan augmentasi rotasi
augment_dataset_with_rotation(input_folder, output_folder)

Augmentasi kelas: Busuk


Kelas: Busuk: 100%|██████████| 159/159 [00:02<00:00, 75.27it/s]


Augmentasi kelas: Matang


Kelas: Matang: 100%|██████████| 160/160 [00:02<00:00, 68.44it/s]


Augmentasi kelas: Mentah


Kelas: Mentah: 100%|██████████| 160/160 [00:02<00:00, 68.19it/s]


Augmentasi kelas: Menuju Matang


Kelas: Menuju Matang: 100%|██████████| 160/160 [00:02<00:00, 64.27it/s]


Augmentasi kelas: Terlalu Matang


Kelas: Terlalu Matang: 100%|██████████| 160/160 [00:02<00:00, 69.17it/s]


**3. Saturation**

In [9]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Fungsi untuk menyesuaikan saturasi gambar
def adjust_saturation(image, saturation_scale):
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Scale the saturation channel
    hsv_image[:, :, 1] = hsv_image[:, :, 1] * saturation_scale
    
    # Ensure values are within [0, 255]
    hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1], 0, 255)
    
    # Convert back to RGB color space
    adjusted_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
    
    return adjusted_image

# Fungsi untuk melakukan augmentasi saturasi pada dataset
def augment_dataset_with_saturation(input_folder, output_folder, desaturation_scale=0.5, saturation_scale=1.5):
    # Pastikan folder output ada atau buat jika belum ada
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Daftar nama kelas
    classes = os.listdir(input_folder)
    
    # Loop untuk setiap kelas
    for class_name in classes:
        print(f"Augmentasi kelas: {class_name}")
        class_folder = os.path.join(input_folder, class_name)
        output_class_folder = os.path.join(output_folder, class_name)
        
        # Buat folder kelas di folder output jika belum ada
        if not os.path.exists(output_class_folder):
            os.makedirs(output_class_folder)
        
        # Daftar semua gambar di folder kelas
        images = os.listdir(class_folder)
        
        # Loop untuk setiap gambar
        for img_name in tqdm(images, desc=f"Kelas: {class_name}", position=0, leave=True):
            # Baca gambar
            img_path = os.path.join(class_folder, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Konversi BGR ke RGB
            
            # Simpan gambar asli
            output_img_path = os.path.join(output_class_folder, img_name)
            cv2.imwrite(output_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
            
            # Augmentasi dengan mengurangi saturasi
            desaturated_img = adjust_saturation(img, desaturation_scale)
            output_img_name = os.path.splitext(img_name)[0] + f"_desaturated.jpg"
            output_img_path = os.path.join(output_class_folder, output_img_name)
            
            # Simpan gambar yang telah dikurangi saturasinya
            cv2.imwrite(output_img_path, cv2.cvtColor(desaturated_img, cv2.COLOR_RGB2BGR))
            
            # Augmentasi dengan meningkatkan saturasi
            saturated_img = adjust_saturation(img, saturation_scale)
            output_img_name = os.path.splitext(img_name)[0] + f"_saturated.jpg"
            output_img_path = os.path.join(output_class_folder, output_img_name)
            
            # Simpan gambar yang telah ditingkatkan saturasinya
            cv2.imwrite(output_img_path, cv2.cvtColor(saturated_img, cv2.COLOR_RGB2BGR))

# Path ke folder dataset asli dan folder output
input_folder = './Dataset/for CNN/train 224'  
output_folder = './Dataset/for CNN/Saturasi 224'  

# Panggil fungsi untuk melakukan augmentasi saturasi
augment_dataset_with_saturation(input_folder, output_folder)


Augmentasi kelas: Busuk


Kelas: Busuk: 100%|██████████| 111/111 [00:02<00:00, 48.81it/s]


Augmentasi kelas: Matang


Kelas: Matang: 100%|██████████| 112/112 [00:01<00:00, 62.90it/s]


Augmentasi kelas: Mentah


Kelas: Mentah: 100%|██████████| 112/112 [00:02<00:00, 45.96it/s]


Augmentasi kelas: Menuju Matang


Kelas: Menuju Matang: 100%|██████████| 112/112 [00:02<00:00, 40.63it/s]


Augmentasi kelas: Terlalu Matang


Kelas: Terlalu Matang: 100%|██████████| 112/112 [00:02<00:00, 53.94it/s]


**4. Splitted CNN**

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_dataset(input_folder, train_folder, val_folder, test_folder, train_size=0.7, val_size=0.2, test_size=0.1, random_state=42):
    # Buat folder train, val, dan test jika belum ada
    for folder in [train_folder, val_folder, test_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    # Daftar nama kelas
    classes = os.listdir(input_folder)
    
    # Loop untuk setiap kelas
    for class_name in classes:
        print(f"Memproses kelas: {class_name}")
        class_folder_input = os.path.join(input_folder, class_name)
        class_folder_train = os.path.join(train_folder, class_name)
        class_folder_val = os.path.join(val_folder, class_name)
        class_folder_test = os.path.join(test_folder, class_name)
        
        # Buat folder kelas di folder train, val, dan test jika belum ada
        for folder in [class_folder_train, class_folder_val, class_folder_test]:
            if not os.path.exists(folder):
                os.makedirs(folder)
        
        # Daftar semua gambar di folder kelas input
        images = os.listdir(class_folder_input)
        
        # Bagi dataset menjadi train dan temp (val + test)
        train_images, temp_images = train_test_split(images, train_size=train_size, random_state=random_state)
        
        # Bagi dataset temp menjadi val dan test
        relative_val_size = val_size / (val_size + test_size)
        val_images, test_images = train_test_split(temp_images, test_size=1 - relative_val_size, random_state=random_state)
        
        # Pindahkan gambar ke folder train, val, dan test
        for img_name in train_images:
            src = os.path.join(class_folder_input, img_name)
            dst = os.path.join(class_folder_train, img_name)
            shutil.copyfile(src, dst)
        
        for img_name in val_images:
            src = os.path.join(class_folder_input, img_name)
            dst = os.path.join(class_folder_val, img_name)
            shutil.copyfile(src, dst)
        
        for img_name in test_images:
            src = os.path.join(class_folder_input, img_name)
            dst = os.path.join(class_folder_test, img_name)
            shutil.copyfile(src, dst)
            
        print(f"Kelas {class_name} dibagi menjadi {len(train_images)} gambar untuk train, {len(val_images)} gambar untuk val, dan {len(test_images)} gambar untuk test.")

# Panggil fungsi untuk membagi dataset
input_folder = "./Dataset/Default"
train_folder = "./Dataset/for CNN/train"
val_folder = "./Dataset/for CNN/val"
test_folder = "./Dataset/for CNN/test"
split_dataset(input_folder, train_folder, val_folder, test_folder, train_size=0.7, val_size=0.2, test_size=0.1, random_state=42)


Memproses kelas: Busuk
Kelas Busuk dibagi menjadi 111 gambar untuk train, 32 gambar untuk val, dan 16 gambar untuk test.
Memproses kelas: Matang
Kelas Matang dibagi menjadi 112 gambar untuk train, 32 gambar untuk val, dan 16 gambar untuk test.
Memproses kelas: Mentah
Kelas Mentah dibagi menjadi 112 gambar untuk train, 32 gambar untuk val, dan 16 gambar untuk test.
Memproses kelas: Menuju Matang
Kelas Menuju Matang dibagi menjadi 112 gambar untuk train, 32 gambar untuk val, dan 16 gambar untuk test.
Memproses kelas: Terlalu Matang
Kelas Terlalu Matang dibagi menjadi 112 gambar untuk train, 32 gambar untuk val, dan 16 gambar untuk test.


**5. Splitted KNN**

In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path ke direktori utama yang berisi folder untuk setiap kelas
main_dir = "./Dataset/1. Resize/"

# List semua kelas
classes = os.listdir(main_dir)
print("Classes:", classes)

# Path untuk menyimpan data latih dan validasi
train_dir = "./Dataset/for KNN/train"
val_dir = "./Dataset/for KNN/val"

# Buat folder train dan val jika belum ada
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Loop untuk setiap kelas
for class_name in classes:
    class_dir = os.path.join(main_dir, class_name)
    images = os.listdir(class_dir)
    
    # Bagi data menjadi train dan val
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Buat folder kelas di train dan val jika belum ada
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    
    if not os.path.exists(train_class_dir):
        os.makedirs(train_class_dir)
    if not os.path.exists(val_class_dir):
        os.makedirs(val_class_dir)
    
    # Copy gambar ke folder train
    for img in train_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(train_class_dir, img)
        shutil.copyfile(src, dst)
    
    # Copy gambar ke folder val
    for img in val_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(val_class_dir, img)
        shutil.copyfile(src, dst)

print("Data train dan val telah dibagi dan disimpan di", train_dir, "dan", val_dir)

Classes: ['Busuk', 'Matang', 'Mentah', 'Menuju Matang', 'Terlalu Matang']
Data train dan val telah dibagi dan disimpan di ./Dataset/for KNN/train dan ./Dataset/for KNN/val
